# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist_name','user_name','gender','item_in_session','user_last_name','length',\
                'level','location','session_id','song','user_id'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# create keyspace if not exists
create_keyspace_command = """CREATE KEYSPACE IF NOT EXISTS sparkify
                             WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""

try:
    session.execute(create_keyspace_command)
except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# Set KEYSPACE to the keyspace specified above
set_keyspace_command = "USE sparkify"

try:
    session.execute(set_keyspace_command)
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




-----------

## 1st Query

### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

We need to select the artist name, song title and song length from song_data_log table, and filter the data by session_id and session_item_number. During the execution our query will looks like:: 
<code>SELECT artist, song_title, song_length FROM session_songs WHERE session_id = 338 AND session_item_number = 4</code>. 
Next steps will describe how to do it: 
* Create table song_data_log if not exists with necessary columns for query.
* Our PRIMARY KEY is composed by session_id (partition key), session_item_number (clustering key) so that we can filter by this attributes later on.
* Load csv file event_datafile_new.csv and insert the data into table song_data_log.
* Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## Create table song_data_log if not exists with necessary columns
create_statement =  """CREATE TABLE IF NOT EXISTS song_data_log (session_id INT,
        session_item_number INT,
        artist_name TEXT,
        song_title TEXT,
        song_length DOUBLE,
        PRIMARY KEY ((session_id, session_item_number)))"""
session.execute(create_statement)

In [ ]:
# Load CSV file data into the table!
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:     
        query = """INSERT INTO song_data_log (session_id, session_item_number, artist_name, song_title, song_length) """
        query = query + "VALUES  (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [ ]:
# SELECT to verify that the data have been inserted into each table
select_statement = "SELECT artist_name, song_title, song_length FROM song_data_log WHERE session_id = %s AND session_item_number =  %s"

result_set = session.execute(select_statement, (338, 4))

for row in result_set:
    print (row)

--------------

## 2nd Query

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

We need to select the artist name, song title and song length from table song_user_data, and filter the data by user_id and session_item_number. During the execution our query will looks like: <code> SELECT artist_name, song_title, user_first_name, user_last_name FROM song_user_data WHERE user_id = 10 AND session_id = 182 </code>. Next steps will describe how to do it: 
* Create table song_data_log if not exists with necessary columns for query.
* Our PRIMARY KEY will consist of composite partition key user_id, session_id. Why, if we only put user_id as partition key, session_id which belongs to the same user will be put into different nodes. That will cause preformance issue with large volume of data.
* Our clustering key will be session_item_number so that our results are order by it.
* Load csv file event_datafile_new.csv and insert the data into table song_user_data.
* Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## Create table song_user_data if not exists with necessary columns
create_statement =  """CREATE TABLE IF NOT EXISTS song_user_data (user_id INT,
        session_id INT,
        session_item_number INT,
        artist_name TEXT,
        song_title TEXT,
        user_first_name TEXT,
        user_last_name TEXT,
        PRIMARY KEY ((user_id, session_id), session_item_number))"""
session.execute(create_statement)         

In [ ]:
# Load CSV file data into the table!
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:        
        query = """INSERT INTO song_user_data (user_id, session_id, session_item_number, artist_name, song_title, user_first_name, user_last_name ) """
        query = query + " VALUES  (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],line[1], line[4]))      

In [ ]:
# SELECT to verify that the data have been inserted into each table
select_statement = "SELECT artist_name, song_title, user_first_name, user_last_name FROM song_user_data WHERE user_id = %s AND session_id = %s"

result_set = session.execute(select_statement, (10, 182))

for row in result_set:
    print (row)    

------------------

## 3rd Query

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

We need to select the user first name and last name from table user_data_song_listened, and filter the data by song_title. During the execution our query will looks like: <code> SELECT user_first_name, user_last_name FROM user_data_song_listened WHERE song_title = 'All Hands Against His Own' </code>. Next steps will describe how to do it:

* Create table user_data_song_listened if not exists with necessary columns for query.
* Our PRIMARY KEY will consist of partition key song_title and clustering key user_id , which uniquely identifies our rows.
* Load csv file event_datafile_new.csv and insert the data into table song_user_data.
* Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## Create table song_user_data if not exists with necessary columns
create_statement =  """CREATE TABLE IF NOT EXISTS user_data_song_listened (song_title TEXT, user_id INT,
        user_first_name TEXT,
        user_last_name TEXT,
        PRIMARY KEY ((song_title), user_id))"""
session.execute(create_statement)         

In [ ]:
# Load CSV file data into the table!
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:   
        query = """INSERT INTO user_data_song_listened (song_title, user_id, user_first_name, user_last_name) """
        query = query + " VALUES  (%s, %s, %s, %s)"
        song_title, user_id, user_first_name, user_last_name = line[9], int(line[10]), line[1], line[4]
        session.execute(query, (song_title, user_id, user_first_name, user_last_name))

In [ ]:
# SELECT to verify that the data have been inserted into each table
select_statement = "SELECT user_first_name, user_last_name FROM user_data_song_listened WHERE song_title = %s"

result_set = session.execute(select_statement, ('All Hands Against His Own', ))

for row in result_set:
    print (row)     

-----------

### Drop the tables before closing out the sessions

In [ ]:
song_data_log =  "DROP TABLE song_data_log"
song_user_data = "DROP TABLE song_user_data"
user_data_by_song_title = "DROP TABLE user_data_song_listened"

session.execute(song_data_log)
session.execute(song_user_data)
session.execute(user_data_by_song_title)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()